In [0]:
import numpy as np
import tensorflow as tf

In [6]:
class HMM(object):
    def __init__(self, initial_prob, trans_prob, obs_prob):
        self.N = np.size(initial_prob)
        self.initial_prob = initial_prob
        self.trans_prob = trans_prob
        self.emission = tf.constant(obs_prob)

        
        self.obs_idx = tf.placeholder(tf.int32)
        self.fwd = tf.placeholder(tf.float64)

    def get_emission(self, obs_idx):
        slice_location = [0, obs_idx]
        num_rows = tf.shape(self.emission)[0]
        slice_shape = [num_rows, 1]#num_hidden_states X 1
        return tf.transpose(tf.slice(self.emission, slice_location, slice_shape))#1 X num_hidden_states

    def forward_init_op(self):
        obs_prob = self.get_emission(self.obs_idx)#1 X N
        fwd = tf.multiply(self.initial_prob, obs_prob)#1 X N mul 1 X N
        #initial_prob - 1 X num_hidden_states obs_probs is also 1 X num_hidden_states
        #fwd = tf.transpose(fwd)
        return fwd

    def forward_op(self):
        tensorProbHiddenStates = tf.matmul(self.fwd,self.trans_prob)#matmul 1 X num_hidden_states to num_hidden_states X num_hidden_states - 1 X num_hidden_states result
        fwd = tensorProbHiddenStates * self.get_emission(self.obs_idx)#1 X num_hidden_states * 1 X num_hidden_states - 1 X num_hidden_states
        return fwd


def forward_algorithm(sess, hmm, observations):
  fwd = sess.run(hmm.forward_init_op(), feed_dict={hmm.obs_idx: observations[0]})
  for t in range(1, len(observations)):
    fwd = sess.run(hmm.forward_op(), feed_dict={hmm.obs_idx: observations[t], hmm.fwd: fwd})
  prob = sess.run(tf.reduce_sum(fwd))
  return prob

if __name__ == '__main__':
    initial_prob = np.array([[0.6], [0.4]])#num_hidden_states X 1
    initial_prob = tf.transpose(initial_prob)#1 X num_hidden_states
    trans_prob = np.array([[0.7, 0.3], [0.4, 0.6]]) # num_hidden_states X num_hidden_states
    obs_prob = np.array([[0.5, 0.4, 0.1], [0.1, 0.3, 0.6]]) # num_hidden_states X num_observables

    hmm = HMM(initial_prob=initial_prob, trans_prob=trans_prob, obs_prob=obs_prob)

    #observations = [0, 1, 1, 2, 1]
    observations = [0, 1, 1, 2, 1, 0, 0, 0, 1, 2, 2, 1]
    with tf.Session() as sess:
        prob = forward_algorithm(sess, hmm, observations)
        print('Probability of observing {} is {}'.format(observations, prob))

Probability of observing [0, 1, 1, 2, 1, 0, 0, 0, 1, 2, 2, 1] is 2.7176685317787254e-06
